In [1]:
from sklearn.multioutput import MultiOutputClassifier
import catboost


In [2]:
#  |  >>> import numpy as np
#  |  >>> from sklearn.datasets import make_multilabel_classification
#  |  >>> from sklearn.multioutput import MultiOutputClassifier
#  |  >>> from sklearn.neighbors import KNeighborsClassifier
#  |  
#  |  >>> X, y = make_multilabel_classification(n_classes=3, random_state=0)
#  |  >>> clf = MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)
#  |  >>> clf.predict(X[-2:])
#  |  array([[1, 1, 0], [1, 1, 1]])

In [3]:
help(MultiOutputClassifier)

Help on class MultiOutputClassifier in module sklearn.multioutput:

class MultiOutputClassifier(sklearn.base.ClassifierMixin, _MultiOutputEstimator)
 |  MultiOutputClassifier(estimator, *, n_jobs=None)
 |  
 |  Multi target classification
 |  
 |  This strategy consists of fitting one classifier per target. This is a
 |  simple strategy for extending classifiers that do not natively support
 |  multi-target classification
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object
 |      An estimator object implementing :term:`fit`, :term:`score` and
 |      :term:`predict_proba`.
 |  
 |  n_jobs : int or None, optional (default=None)
 |      The number of jobs to run in parallel.
 |      :meth:`fit`, :meth:`predict` and :meth:`partial_fit` (if supported
 |      by the passed estimator) will be parallelized for each target.
 |  
 |      When individual estimators are fast to train or predict,
 |      using ``n_jobs > 1`` can result in slower performance due
 |      to the para

# https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.ClassifierChain.html#sklearn.multioutput.ClassifierChain

In [56]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [76]:
df_train = pd.read_csv('train_go05W65.csv')
# df = X.copy()
df_test = pd.read_csv('test_VkM91FT.csv')
tst = pd.read_csv('test_VkM91FT.csv')
SAM = pd.read_csv('sample_submission_kF044ur.csv')

In [58]:
df_train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2
0,CC264719,Male,41,14,0,C1,S3,['P16'],['P8']
1,CC209679,Female,47,14,1,C1,S2,"['P13', 'P20']",['P3']
2,CC319633,Female,59,14,0,C2,S2,['P11'],['P00']
3,CC231413,Female,32,16,0,C1,S2,"['P8', 'P13']",['P6']
4,CC259633,Male,30,15,0,C2,S3,"['P16', 'P17', 'P21']","['P8', 'P12']"


In [59]:
df_test.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1
0,CC372708,Female,31,31,0,C2,S3,"['P12', 'P13']"
1,CC216072,Male,28,37,1,C1,S2,"['P12', 'P13']"
2,CC387629,Male,31,12,0,C2,S3,['P20']
3,CC389228,Female,55,11,0,C2,S2,"['P13', 'P21']"
4,CC394445,Male,51,49,1,C2,S1,['P13']


In [60]:
SAM.head()
# type(SAM.head()['Product_Holding_B2'][1])  #--->str dtype

,Customer_ID,Product_Holding_B2
0,CC372708,['P00']
1,CC216072,['P00']
2,CC387629,['P00']
3,CC389228,['P00']
4,CC394445,['P00']


In [61]:
type(df_train.Product_Holding_B1[1])

str

# converting string type to list type columns

In [62]:
df_train['Product_Holding_B1']= df_train.Product_Holding_B1.apply(eval) 
df_train['Product_Holding_B2']= df_train.Product_Holding_B2.apply(eval) 

In [63]:
df_test['Product_Holding_B1']= df_test.Product_Holding_B1.apply(eval) 

In [64]:
df_train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8]
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3]
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00]
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6]
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]"


In [65]:
df_test.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1
0,CC372708,Female,31,31,0,C2,S3,"[P12, P13]"
1,CC216072,Male,28,37,1,C1,S2,"[P12, P13]"
2,CC387629,Male,31,12,0,C2,S3,[P20]
3,CC389228,Female,55,11,0,C2,S2,"[P13, P21]"
4,CC394445,Male,51,49,1,C2,S1,[P13]


In [66]:
df_test.Product_Holding_B1[1]

['P12', 'P13']

# Dealing with list columns

<!-- Dealing with List columns -->

https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
convert Target Variables to list while loading the csv

from ast import literal_eval
df = pd.read_csv('test.csv', converters={'col1': literal_eval})

as mentioned in below link:
https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list/23112008

following which you can one hot encode the target variable using the method mentioned in the following link:
https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list    
 
https://realpython.com/python-eval-function/#minimizing-the-security-issues-of-eval

https://it-qa.com/how-do-you-predict-multi-label-classification/


https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/

In [67]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer(sparse_output=True)

# df = df.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 mlb.fit_transform(df.pop('Col3')),
#                 index=df.index,
#                 columns=mlb.classes_))

In [68]:
#  split_df = pd.get_dummies(df.Label.str.split(" ").explode())        


# Series.str.split(expand)#test

In [77]:
tst.Product_Holding_B1.head()

0    ['P12', 'P13']
1    ['P12', 'P13']
2           ['P20']
3    ['P13', 'P21']
4           ['P13']
Name: Product_Holding_B1, dtype: object

In [78]:
type(tst.Product_Holding_B1[0])

str

In [140]:
print(type(pd.DataFrame(tst.Product_Holding_B1)))
# pd.DataFrame(tst.Product_Holding_B1).replace([r'^\[',r'\]$'],r'',regex=True)
import re
pd.DataFrame(tst.Product_Holding_B1).replace([r'^\[',r'\]$'],r'',regex=True).apply(lambda x: x.str.split(","),axis=1).loc[0]

<class 'pandas.core.frame.DataFrame'>


Product_Holding_B1    ['P12',  'P13']
Name: 0, dtype: object

In [97]:
# tst.Product_Holding_B1.str.split(',',expand=True)
# tst.Product_Holding_B1.replace('[','').replace(']','').str.split(',',expand=True)
tst.Product_Holding_B1.str.replace('[','').str.replace(']','').str.split(',',expand=True)

,0,1,2,3,4,5,6,7
0,'P12','P13',None,None,None,None,None,None
1,'P12','P13',None,None,None,None,None,None
2,'P20',None,None,None,None,None,None,None
3,'P13','P21',None,None,None,None,None,None
4,'P13',None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
20322,'P16',None,None,None,None,None,None,None
20323,'P16',None,None,None,None,None,None,None
20324,'P13',None,None,None,None,None,None,None
20325,'P8','P9','P16',None,None,None,None,None


In [17]:
df_train.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2'],
      dtype='object')

In [18]:
df_train.isnull().sum()

Customer_ID           0
Gender                0
Age                   0
Vintage               0
Is_Active             0
City_Category         0
Customer_Category     0
Product_Holding_B1    0
Product_Holding_B2    0
dtype: int64

In [19]:
df_test.isnull().sum()

Customer_ID           0
Gender                0
Age                   0
Vintage               0
Is_Active             0
City_Category         0
Customer_Category     0
Product_Holding_B1    0
dtype: int64

In [20]:
df_train.columns

Index(['Customer_ID', 'Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'Product_Holding_B1', 'Product_Holding_B2'],
      dtype='object')

In [21]:
df_train[['Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category','Customer_Category']].nunique()

Gender                2
Age                  36
Vintage              64
Is_Active             2
City_Category         2
Customer_Category     3
dtype: int64

In [22]:
df_train.head()

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1,Product_Holding_B2
0,CC264719,Male,41,14,0,C1,S3,[P16],[P8]
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]",[P3]
2,CC319633,Female,59,14,0,C2,S2,[P11],[P00]
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]",[P6]
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]","[P8, P12]"


In [23]:
df_test[['Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category','Customer_Category']].nunique()

Gender                2
Age                  36
Vintage              63
Is_Active             2
City_Category         2
Customer_Category     3
dtype: int64

In [24]:
df_train.Product_Holding_B2

0             [P8]
1             [P3]
2            [P00]
3             [P6]
4        [P8, P12]
           ...    
37743         [P9]
37744    [P5, P12]
37745        [P13]
37746         [P1]
37747    [P8, P12]
Name: Product_Holding_B2, Length: 37748, dtype: object

In [25]:
# ---------

In [26]:
y = df_train['Product_Holding_B2']
X = df_train.drop('Product_Holding_B2',axis=1)

In [27]:
X

,Customer_ID,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,Product_Holding_B1
0,CC264719,Male,41,14,0,C1,S3,[P16]
1,CC209679,Female,47,14,1,C1,S2,"[P13, P20]"
2,CC319633,Female,59,14,0,C2,S2,[P11]
3,CC231413,Female,32,16,0,C1,S2,"[P8, P13]"
4,CC259633,Male,30,15,0,C2,S3,"[P16, P17, P21]"
...,...,...,...,...,...,...,...,...
37743,CC314217,Male,28,24,1,C1,S2,[P16]
37744,CC388747,Male,36,11,0,C2,S2,"[P16, P17, P21]"
37745,CC318056,Male,44,26,1,C2,S2,[P14]
37746,CC373551,Female,56,15,1,C1,S2,"[P12, P13]"


In [28]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

X = X.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(X['Product_Holding_B1']),
                index=X.index,
                columns=mlb.classes_))

 
df_test = df_test.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df_test['Product_Holding_B1']),
                index=df_test.index,
                columns=mlb.classes_))



In [29]:
X.City_Category.value_counts()

C1    18941
C2    18807
Name: City_Category, dtype: int64

In [30]:
X.Customer_Category.value_counts()

S3    17865
S2    13265
S1     6618
Name: Customer_Category, dtype: int64

In [31]:
X.Gender = X.Gender.map({'Male':0,'Female':1})
X.City_Category = X.City_Category.map({'C1':0,'C2':1})
X.Customer_Category = X.Customer_Category.map({'S1':0,'S2':1,'S3':2})

In [32]:
df_test.Gender = df_test.Gender.map({'Male':0,'Female':1})
df_test.City_Category = df_test.City_Category.map({'C1':0,'C2':1})
df_test.Customer_Category = df_test.Customer_Category.map({'S1':0,'S2':1,'S3':2})

In [33]:
X = X.drop(['Product_Holding_B1','Customer_ID'],axis=1)
df_test = df_test.drop(['Product_Holding_B1','Customer_ID'],axis=1)
X.nunique()

Gender                2
Age                  36
Vintage              64
Is_Active             2
City_Category         2
Customer_Category     3
P00                   2
P1                    2
P10                   2
P11                   2
P12                   2
P13                   2
P14                   2
P15                   2
P16                   2
P17                   2
P18                   2
P19                   2
P2                    2
P20                   2
P21                   2
P3                    2
P4                    2
P5                    2
P6                    2
P7                    2
P8                    2
P9                    2
dtype: int64

In [34]:
X.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,P00,P1,P10,P11,...,P2,P20,P21,P3,P4,P5,P6,P7,P8,P9
0,0,41,14,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,47,14,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,59,14,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,32,16,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,30,15,0,1,2,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [35]:
from sklearn.preprocessing import StandardScaler


# Apply StandardScaler to parts of a data set

# https://stackoverflow.com/questions/38420847/apply-standardscaler-to-parts-of-a-data-set

In [36]:
scaled_features = X.copy()

In [37]:
col_names = ['Age', 'Vintage']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df_test[col_names] = scaler.transform(df_test[col_names].values)
scaled_features[col_names] = features

X = scaled_features

In [38]:
X.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,P00,P1,P10,P11,...,P2,P20,P21,P3,P4,P5,P6,P7,P8,P9
0,0,0.244882,-0.543603,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0.837670,-0.543603,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,2.023244,-0.543603,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,-0.644298,-0.348524,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,-0.841894,-0.446064,0,1,2,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [39]:
df_test.head()

,Gender,Age,Vintage,Is_Active,City_Category,Customer_Category,P00,P1,P10,P11,...,P2,P20,P21,P3,P4,P5,P6,P7,P8,P9
0,1,-0.743096,1.114565,0,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,-1.039490,1.699800,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,-0.743096,-0.738681,0,1,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1.628053,-0.836221,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1.232861,2.870271,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
type(y[1])

list

In [41]:
# y = pd.DataFrame(y).reset_index()

In [42]:
y = pd.DataFrame(y)
y

,Product_Holding_B2
0,[P8]
1,[P3]
2,[P00]
3,[P6]
4,"[P8, P12]"
...,...
37743,[P9]
37744,"[P5, P12]"
37745,[P13]
37746,[P1]


In [43]:
y=y.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(y['Product_Holding_B2']),
                index=y.index,
                columns=mlb.classes_))

y = y.drop("Product_Holding_B2",axis=1)
    

In [44]:
type(y.values)

numpy.ndarray

In [45]:
(np.array(y)).shape

(37748, 20)

In [46]:
# from scipy.sparse import csr_matrix
# y = csr_matrix(y)

In [47]:
y

,P00,P1,P10,P11,P12,P13,P14,P15,P16,P17,P18,P2,P20,P3,P4,P5,P6,P7,P8,P9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37743,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
37744,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
37745,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37746,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# param_grid = {"n_estimators":[1,5,10,20,40,100],'max_depth':[3,4,5,6]}
# gb_model = GradientBoostingClassifier()
# grid = GridSearchCV(gb_model,param_grid)

In [49]:
# class skmultilearn.problem_transform.LabelPowerset(classifier=None, require_dense=None)

In [50]:
y.shape



(37748, 20)

In [51]:
# dir(LabelPowerset)

## Adaptive method

In [52]:
# from skmultilearn.adapt import MLkNN

# classifier = MLkNN(k=20)

# # train
# classifier.fit(X,y.values)

In [53]:
# run form below for models

## labelset model

In [54]:
lablmodel(X,y)#labelset model
# lbm_xgb(X,y)

NameError: name 'lablmodel' is not defined

In [ ]:
# clf.best_params_

In [ ]:
predictions = classifier.predict(df_test)
# predictions = clf.predict(df_test)

In [ ]:
s = predictions.toarray()

## https://xang1234.github.io/multi-label/

https://nickcdryan.com/2017/01/23/multi-label-classification-a-guided-tour/

In [ ]:
s

In [ ]:
pred  = pd.DataFrame(s,columns =y.columns) 

In [ ]:
# y.columns
# # Index(['P00', 'P1', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17',
# #        'P18', 'P2', 'P20', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'],
# #       dtype='object')

In [ ]:
pred.head()

In [ ]:
# pred.where()

In [ ]:
pred.loc[200]

In [ ]:
# pred.where(pred == 1)

In [ ]:
# ks = []
ss = []
for k,v in pred.iteritems():
#     m.append(k:v.map({1:k}))
#     print("k:",k)
#     ks.append(k)#not required as series object itself has name
#     print("ser:",v.map({1:v.name,0:0}))
#     ss.append(v.map({1:k,0:0}))
    ss.append(v.map({1:v.name,0:0})

In [ ]:
# len(ks)

In [ ]:
ss[0].name, ss[0]

In [ ]:
final = pd.concat(ss,axis=1)

In [ ]:
final.head()

In [ ]:
n =pd.DataFrame(final.apply(lambda x: [i for i in x if i in final.columns],axis=1))

In [ ]:
output = pd.concat([tst.Customer_ID,n],axis=1)

In [ ]:
output.columns = ["Customer_ID", "Product_Holding_B2"]

In [ ]:
output.Product_Holding_B2 = output.Product_Holding_B2.astype('str')

In [ ]:
output.head()

In [ ]:
output.to_csv('submission2.csv', index=False)

In [ ]:
SAM.Product_Holding_B2[0]

In [ ]:
SAM

In [ ]:
output

## Testing

In [ ]:
[i for i in final.loc[200] if i in final.columns]

In [ ]:
final.iloc[200] 

In [ ]:
type(final.iloc[0][-1])

In [ ]:
l = pd.DataFrame({'k':[1,2,1,3,4],'j':[1,2,1,3,4]})

In [ ]:
pd.concat([m,l])

In [ ]:
h  = pd.DataFrame(pd.Series({'p1':0,'p2':1,'p3':1})).T
h

In [ ]:
h = pd.concat([h,h])
h =h.reset_index(drop=True)

In [ ]:
pd.Series((1,1,0,1,1)).map({1:2})

In [ ]:
for key, value in h.iteritems():
    print('Key is :', key)
    print('Value is:\n', value.map({1:key,0:0}), '\n')

In [ ]:
y.values

In [ ]:
y.head()

In [ ]:
# from scipy.sparse import csr_matrix
# y = csr_matrix(y)

# Models

In [ ]:
def lablmodel(X,y):
    from skmultilearn.problem_transform import LabelPowerset
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.naive_bayes import GaussianNB

    # initialize Label Powerset multi-label classifier
    # with a gaussian naive bayes base classifier
    classifier = LabelPowerset(GradientBoostingClassifier(), require_dense=[True, True])
    # classifier = LabelPowerset(GaussianNB())
    # classifier = LabelPowerset(GaussianNB, require_dense=[False, True])

    # train
    # classifier.fit(X,y.values.reshape(-1, 1))
    classifier.fit(X,y)


    # predict
    # predictions = classifier.predict(df_test)


In [ ]:
def lbm_xgb(X,y):
    from skmultilearn.problem_transform import LabelPowerset
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import GridSearchCV
    # initialize Label Powerset multi-label classifier
    # with a gaussian naive bayes base classifier
    parameters = [
    
    {
        'classifier': [GradientBoostingClassifier()],
        'classifier__n_estimators': [1,5,10,20,40,100],
        'classifier__max_depth':[3,4,5,6]
    },
    ]

    clf = GridSearchCV(LabelPowerset(), parameters, scoring='accuracy')
    clf.fit(X, y)
    
#     gb_model = GradientBoostingClassifier()
#     param_grid = {"gb_model__n_estimators":[1,5,10,20,40,100],'gb_model__max_depth':[3,4,5,6]}
#     classifier = LabelPowerset(GaussianNB(), require_dense=[True, True])
    
    # classifier = LabelPowerset(GaussianNB())
    # classifier = LabelPowerset(GaussianNB, require_dense=[False, True])

    # train
    # classifier.fit(X,y.values.reshape(-1, 1))
#     classifier.fit(X,y)
#     clf = GridSearchCV(LabelPowerset(), parameters, scoring='accuracy')
#     clf.fit(x, y)


    # predict
    # predictions = classifier.predict(df_test)


In [ ]:
#  split_df = pd.get_dummies(df.Product_Holding_B2.str.replace('[','').str.replace(']','').str.split(",").explode())
# split_df.columns

# Notes: